<a href="https://colab.research.google.com/github/javier-jaime/Tool-Crib/blob/master/Colab/SVRP_PYOMO_AbstractModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Stochastic Optimization for Vehicle Routing Problem with PYOMO

### Import Necessary Libraries

In [ ]:
!pip install pyomo
!sudo apt-get install glpk-utils

from pyomo.environ import *
from pyomo import environ as pe
# from coopr.pyomo import *
#from coopr.opt.base import solver
from pyomo.opt import *

from google.colab import drive
from google.colab import files

import scipy
import numpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Reading package lists... Done
Building dependency tree       
Reading state information... Done
glpk-utils is already the newest version (4.65-1).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'sudo apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 4 not upgraded.


In [ ]:
# Mount Google Drive

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Import Data File
!cp drive/MyDrive/Colab_Data/svrp_data.dat "/content/"

### Abstract Model

In [ ]:
# AbstractModel is where data values are supplied in a data file

model = AbstractModel()

In [ ]:
# Define sets

model.I = Set() #node
model.J = Set()
model.S = Set() #source node
model.D = Set() #demand node

In [ ]:
# Data_deterministic

model.Arc = Param(model.I, model.J) #arc available
model.Rev = Param(model.I, model.J) #arc revenue
model.Cost = Param(model.I, model.J) #arc cost
model.B = Param()

In [ ]:
#Data_stochastic

model.ArcDemand = Param(model.I, model.J) #arc demand

In [ ]:
# Variables

model.X_WS = Param(model.S)
model.X_EV = Param(model.S)
#model.X_RP = Param(initialize=lambda m: model.S[value(model.S)])
model.X = Var(model.S, bounds=(0.0, model.B))
model.Y = Var(model.I, model.J, bounds=(0.0, model.B))
model.Z = Var(model.I, model.J, bounds=(0.0, None))

model.FirstStageProfit = Var()
model.SecondStageProfit = Var()

In [ ]:
# Constraints

def vehicle_num_cap_rule(model):
    return sum(model.X[s] for s in model.S) <= model.B
model.VehicleNumCapRule = Constraint(rule=vehicle_num_cap_rule)


def vehicle_assigned_cap_rule(model,s):
    return sum(model.Y[s,j] for j in model.J if model.Arc[s,j]>=1) == model.X[s]
model.RequiredDemandRule = Constraint(model.S, rule=vehicle_assigned_cap_rule)

def flow_balance_rule(model,d):
    return (sum(model.Y[i,d] for i in model.I if model.Arc[i,d]>=1) - sum(model.Y[d,i] for i in model.I if model.Arc[d,i]>=1)) == 0.0
model.FlowBalanceRule = Constraint(model.D, rule=flow_balance_rule)

def overage_rule(model,i,j):
    return model.Y[i,j] - model.ArcDemand[i,j] <= model.Z[i,j]
model.OverageRule = Constraint(model.I, model.J, rule=overage_rule)

def y_rule(model,i,j):
    return (0.0, model.Y[i,j], model.Arc[i,j]*51)
model.YRule = Constraint(model.I, model.J, rule=y_rule)

#NOTE: (Part H) We have added a constraint to fix X at RP
##def x_fix_rule(model,s):
##    return model.X[s] == model.X_RP[s]
##model.XFixRule = Constraint(model.S, rule=x_fix_rule)

In [ ]:
# Stage-specific cost computations

def first_stage_profit_rule(model):
    return model.FirstStageProfit == 0.0
model.ComputeFirstStageProfit = Constraint(rule=first_stage_profit_rule)

def second_stage_profit_rule(model):
    return model.SecondStageProfit - sum(sum(model.Rev[i,j] * model.Y[i,j] - (model.Rev[i,j] + model.Cost[i,j])* model.Z[i,j] \
                                           for i in model.I) for j in model.J) == 0.0
model.ComputeSecondStageProfit = Constraint(rule=second_stage_profit_rule)

In [ ]:
# Objective

def total_profit_rule(model):
    return (model.FirstStageProfit + model.SecondStageProfit)

model.Total_Profit_Objective = Objective(rule=total_profit_rule, sense=maximize)

In [ ]:
# Solve WS for given number of sample realizations with fixed X at X_WS
numSamples=100
numX=5
optVal=numpy.array([0 for i in range(numSamples)])

In [ ]:
for i in range(numSamples):
    datafile='svrp_data.dat'
    #datafile = '../drive/MyDrive/Colab_Data/Scenario' + str(i+1) + '.dat'
    instance = model.create_instance(datafile)
    opt= pe.SolverFactory("glpk")
    results = opt.solve(instance, tee=True)
    instance.solutions.store_to(results)
    results.write()
    #instance.pprint()
    #instance.display()
    optVal[i] = value(instance.Total_Profit_Objective)

Streaming output truncated to the last 5000 lines.
      Value: 1.5
    Y[13,17]:
      Value: 4.65
    Y[13,18]:
      Value: 1.85
    Y[14,17]:
      Value: 2.05
    Y[14,18]:
      Value: 2.95
    Y[14,19]:
      Value: 1.1
    Y[15,20]:
      Value: 7.15
    Y[16,20]:
      Value: 9.9
    Y[17,20]:
      Value: 10.55
    Y[18,20]:
      Value: 6.7
    Y[19,20]:
      Value: 3.1
    Y[2,11]:
      Value: 2.9
    Y[2,12]:
      Value: 2.05
    Y[2,6]:
      Value: 1.25
    Y[2,7]:
      Value: 4.45
    Y[2,8]:
      Value: 1.05
    Y[3,12]:
      Value: 1.1
    Y[3,7]:
      Value: 0.9
    Y[3,8]:
      Value: 1.25
    Y[3,9]:
      Value: 1.85
    Y[4,10]:
      Value: 2.05
    Y[4,13]:
      Value: 2.1
    Y[4,8]:
      Value: 1.25
    Y[4,9]:
      Value: 2.9
    Y[5,10]:
      Value: 2.05
    Y[5,14]:
      Value: 2.9
    Y[6,12]:
      Value: 1.75
    Y[6,15]:
      Value: 1.5
    Y[6,16]:
      Value: 1.1
    Y[7,11]:
      Value: 1.25
    Y[7,12]:
      Value: 2.1
    Y[7,13]: